<a href="https://colab.research.google.com/github/kridtapon/Consolidation-Reversal-System/blob/main/Consolidation_Reversal_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Gopalakrishnan Range Index (GAPO)
def calculate_gapo(df, period=14):
    """
    Calculate Gopalakrishnan Range Index (GAPO).
    GAPO is calculated as (High - Low) / (High + Low) over a specified period.
    """
    df['Range'] = df['High'] - df['Low']
    df['GAPO'] = (df['Range'].rolling(window=period).sum()) / (df['High'].rolling(window=period).sum() + df['Low'].rolling(window=period).sum())
    return df

# Function to calculate MACD and Signal Line
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD and Signal Line.
    """
    macd = df['Close'].ewm(span=fast_period, adjust=False).mean() - df['Close'].ewm(span=slow_period, adjust=False).mean()
    macd_signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, macd_signal

# Define the stock symbol and time period
symbol = 'DECK'  #
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Gopalakrishnan Range Index (GAPO)
df = calculate_gapo(df, period=14)

# Calculate MACD and Signal Line
df['MACD'], df['MACD_Signal'] = calculate_macd(df)

# Define Entry and Exit signals based on GAPO and MACD
# Entry Signal: GAPO is low (indicating consolidation), MACD line crosses above the signal line.
df['Entry'] = (df['GAPO'] < df['GAPO'].quantile(0.6)) & (df['MACD'] > df['MACD_Signal'])

# Exit Signal: GAPO is low, MACD line crosses below the signal line.
df['Exit'] = (df['GAPO'] > df['GAPO'].quantile(0.9)) & (df['MACD'] < df['MACD_Signal'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           324300.246868
Total Return [%]                       224.300247
Benchmark Return [%]                   607.753959
Max Gross Exposure [%]                      100.0
Total Fees Paid                        903.751656
Max Drawdown [%]                        33.374978
Max Drawdown Duration                       453.0
Total Trades                                    5
Total Closed Trades                             4
Total Open Trades                               1
Open Trade PnL                      218580.281655
Win Rate [%]                                 75.0
Best Trade [%]                          15.969342
Worst Trade [%]                         -11.10852
Avg Winning Trade [%]                    6.170538
Avg Losing Trade [%]                    -11.10852


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from sklearn.model_selection import ParameterGrid

# Function to calculate Gopalakrishnan Range Index (GAPO)
def calculate_gapo(df, period=14):
    """
    Calculate Gopalakrishnan Range Index (GAPO).
    GAPO is calculated as (High - Low) / (High + Low) over a specified period.
    """
    df['Range'] = df['High'] - df['Low']
    df['GAPO'] = (df['Range'].rolling(window=period).sum()) / (df['High'].rolling(window=period).sum() + df['Low'].rolling(window=period).sum())
    return df

# Function to calculate MACD and Signal Line
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD and Signal Line.
    """
    macd = df['Close'].ewm(span=fast_period, adjust=False).mean() - df['Close'].ewm(span=slow_period, adjust=False).mean()
    macd_signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, macd_signal

# Define the stock symbol and time period
symbol = 'AXON'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define the grid for optimization
param_grid = {
    'gapo_period': [10, 14, 20, 25],  # Possible GAPO periods
    'macd_fast_period': [8, 12, 16],  # Possible MACD fast periods
    'macd_slow_period': [18, 26, 30],  # Possible MACD slow periods
    'macd_signal_period': [5, 9, 12],  # Possible MACD signal periods
    'gapo_quantile_entry': [0.1, 0.2, 0.3, 0.4],  # Possible GAPO quantiles for entry
    'gapo_quantile_exit': [0.6, 0.7, 0.8, 0.9]  # Possible GAPO quantiles for exit
}

# Create a parameter grid for all combinations
grid = ParameterGrid(param_grid)

# Store results for each parameter combination
results = []

# Grid search for optimal parameters
for params in grid:
    gapo_period = params['gapo_period']
    fast_period = params['macd_fast_period']
    slow_period = params['macd_slow_period']
    signal_period = params['macd_signal_period']
    gapo_quantile_entry = params['gapo_quantile_entry']
    gapo_quantile_exit = params['gapo_quantile_exit']

    # Calculate GAPO
    df = calculate_gapo(df, period=gapo_period)

    # Calculate MACD and Signal Line
    df['MACD'], df['MACD_Signal'] = calculate_macd(df, fast_period, slow_period, signal_period)

    # Define Entry and Exit signals based on GAPO and MACD
    df['Entry'] = (df['GAPO'] < df['GAPO'].quantile(gapo_quantile_entry)) & (df['MACD'] > df['MACD_Signal'])
    df['Exit'] = (df['GAPO'] > df['GAPO'].quantile(gapo_quantile_exit)) & (df['MACD'] < df['MACD_Signal'])

    # Filter data for the test period (2020-2025)
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Store performance stats for the combination of parameters
    stats = portfolio.stats()
    results.append((params, stats))

# Find the best parameter set based on the maximum Total Return
best_result = max(results, key=lambda x: x[1]['Total Return [%]'])

# Print the best parameters and performance metrics
best_params, best_stats = best_result
print("Best Parameters:", best_params)
print("Best Performance Stats:", best_stats)

# Plot equity curve for the best result
best_params_for_backtest = best_params
df = calculate_gapo(df, period=best_params_for_backtest['gapo_period'])
df['MACD'], df['MACD_Signal'] = calculate_macd(df, best_params_for_backtest['macd_fast_period'], best_params_for_backtest['macd_slow_period'], best_params_for_backtest['macd_signal_period'])
df['Entry'] = (df['GAPO'] < df['GAPO'].quantile(best_params_for_backtest['gapo_quantile_entry'])) & (df['MACD'] > df['MACD_Signal'])
df['Exit'] = (df['GAPO'] > df['GAPO'].quantile(best_params_for_backtest['gapo_quantile_exit'])) & (df['MACD'] < df['MACD_Signal'])

df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
portfolio_best = vbt.Portfolio.from_signals(
    close=df_filtered['Close'],
    entries=df_filtered['Entry'],
    exits=df_filtered['Exit'],
    init_cash=100_000,
    fees=0.001
)
portfolio_best.plot().show()


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: 

Best Parameters: {'gapo_period': 14, 'gapo_quantile_entry': 0.4, 'gapo_quantile_exit': 0.9, 'macd_fast_period': 8, 'macd_signal_period': 5, 'macd_slow_period': 26}
Best Performance Stats: Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           782747.278007
Total Return [%]                       682.747278
Benchmark Return [%]                   676.280076
Max Gross Exposure [%]                      100.0
Total Fees Paid                       1158.633251
Max Drawdown [%]                        25.053205
Max Drawdown Duration                       373.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                      560260.198431
Win Rate [%]                                100.0
Best Trade [